# tf-idf

英文全称： term frequency-inverse document frequency

tf-idf是一种统计方法，用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度。它是一种用于信息检索与文本挖掘的常用加权技术。
$$
tf-idf = tf * idf
		 
$$


其中：
$$
tf = \frac{n}{N} , 即 词频 = \frac{词语在文本中出现的次数}{文本中总词语数目}
$$

$$
idf = log\frac{D}{d}, 即 逆向文档频率 = log\frac{总的文档数}{词语所在的文档数}
$$
对于每个词语来说，总的文档数D都是一样的，只是会出现的不同数量的文档里，如果一个词语出现在多个文档里，idf会变小。如此，tf-idf的值会随着tf的增大而正比增大，但随着它在更多文档中出现而下降。

举一个简单的例子：

假设有一个文本:

```python
train = ['Chinese Beijing Chinese',
         'Chinese Chinese Shanghai',
         'Chinese Macao',
         'Tokyo Tapan Chinese']
```

按照特征 `['beijing' 'chinese' 'macao' 'shanghai' 'tapan' 'tokyo']` 计算第一行文本信息的tf-idf值，输出矩阵如下：
```python
[[0.6931471805599453, 0, ...],
 [...], ...]
```
第一行中，词语 "beijing" 出现了1次，总词语数目为3。那么词语 "beijing" 的TF为：

TF("beijing") = 1 / 3 = 0.33333333333

总的文本行是4, 'beijing'只在第一个文本行出现过, 所以

IDF("beijing") = log(4 / 1) = 1.3862943611198906

最终，TF-IDF("beijing") = 0.33333333333 * 1.3862943611198906

同样，计算第一行中'chinese'的tf-idf值
TF('chinese') = 2 / 3 = 0.66666666667
IDF('chinese') = log(4 / 4) = 0
TF-IDF('chinese') = 0


**不足：**

- 频率越小单词约重要？

- 单词频率越大就越无用？

- 不能体现上下文信息？

In [4]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer

train = ['Chinese Beijing Chinese',
         'Chinese Chinese Shanghai',
         'Chinese Macao',
         'Tokyo Tapan Chinese']

# 计算词频矩阵
count_vectorizer = CountVectorizer()
train_counts = count_vectorizer.fit_transform(train)
print(count_vectorizer.get_feature_names_out())
print(train_counts.toarray())

# 创建TfidfTransformer对象
transformer = TfidfTransformer(norm='l1')
tfidf_matrix = transformer.fit_transform(train_counts)
print(tfidf_matrix.toarray())


['beijing' 'chinese' 'macao' 'shanghai' 'tapan' 'tokyo']
[[1 2 0 0 0 0]
 [0 2 0 1 0 0]
 [0 1 1 0 0 0]
 [0 1 0 0 1 1]]
[[0.48931268 0.51068732 0.         0.         0.         0.        ]
 [0.         0.51068732 0.         0.48931268 0.         0.        ]
 [0.         0.34290134 0.65709866 0.         0.         0.        ]
 [0.         0.20692874 0.         0.         0.39653563 0.39653563]]


# sklearn-tf-idf

在sklearn中，$ tf = n $，并未直接做归一化；$ idf = log\frac{D+1}{d+1} + 1 $，对分母和分子做了平滑处理，这样为了分母不为零。

## norm 设置为None

In [16]:
import os
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances

tv = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None)

# 输入训练集矩阵，每行表示一个文本
train = ['Chinese Beijing Chinese',
         'Chinese Chinese Shanghai',
         'Chinese Macao',
         'Tokyo Tapan Chinese']

# 训练，构建词汇表以及词项idf值，并将文本列表转成VSM矩阵格式
tv_fit = tv.fit_transform(train)

# 查看构建的词汇表
print(tv.get_feature_names_out())

# 将稀疏矩阵转换为稠密矩阵
arrs = tv_fit.toarray()
print(arrs)


['beijing' 'chinese' 'macao' 'shanghai' 'tapan' 'tokyo']
[[1.91629073 2.         0.         0.         0.         0.        ]
 [0.         2.         0.         1.91629073 0.         0.        ]
 [0.         1.         1.91629073 0.         0.         0.        ]
 [0.         1.         0.         0.         1.91629073 1.91629073]]


解释下上述值是怎么计算出来的，以第一行为例
'beijing' 在第一行中出现了一次，所以
TF('beijing') = 1
总的文本行是4个，'beijing'只在第一行出现，所以
IDF('beijing') = log((4+1) / (1+1)) + 1 = log(5/2) + 1 = 1.916290731874155
TF-IDF('beijing') = 1 * 1.916290731874155

TF('chinese') = 2
IDF('chinese') = log((4+1) / (4+1)) + 1 = log(1) + 1 = 1
TF-IDF('chinese') = 2 * 1 = 2

## norm 为l1归一化

In [17]:
tv = TfidfVectorizer(use_idf=True, smooth_idf=True, norm='l1')

# 输入训练集矩阵，每行表示一个文本
train = ['Chinese Beijing Chinese',
         'Chinese Chinese Shanghai',
         'Chinese Macao',
         'Tokyo Tapan Chinese']

# 训练，构建词汇表以及词项idf值，并将文本列表转成VSM矩阵格式
tv_fit = tv.fit_transform(train)

# 查看构建的词汇表
print(tv.get_feature_names_out())

arrs = tv_fit.toarray()
print(arrs)


['beijing' 'chinese' 'macao' 'shanghai' 'tapan' 'tokyo']
[[0.48931268 0.51068732 0.         0.         0.         0.        ]
 [0.         0.51068732 0.         0.48931268 0.         0.        ]
 [0.         0.34290134 0.65709866 0.         0.         0.        ]
 [0.         0.20692874 0.         0.         0.39653563 0.39653563]]


这里的0.48931268 = 1.91629073 / sum(1.91629073, 2, 0, 0, 0, 0) 
另外可以看到，每一行的相加和为1

## norm 为l2归一化

In [19]:
tv = TfidfVectorizer(use_idf=True, smooth_idf=True, norm='l2')

# 输入训练集矩阵，每行表示一个文本
train = ['Chinese Beijing Chinese',
         'Chinese Chinese Shanghai',
         'Chinese Macao',
         'Tokyo Tapan Chinese']

# 训练，构建词汇表以及词项idf值，并将文本列表转成VSM矩阵格式
tv_fit = tv.fit_transform(train)

# 查看构建的词汇表
print(tv.get_feature_names_out())

arrs = tv_fit.toarray()
print(arrs)
arrs = np.power(arrs, 2)
print(arrs)

['beijing' 'chinese' 'macao' 'shanghai' 'tapan' 'tokyo']
[[0.69183461 0.722056   0.         0.         0.         0.        ]
 [0.         0.722056   0.         0.69183461 0.         0.        ]
 [0.         0.46263733 0.88654763 0.         0.         0.        ]
 [0.         0.34618161 0.         0.         0.66338461 0.66338461]]
[[0.47863513 0.52136487 0.         0.         0.         0.        ]
 [0.         0.52136487 0.         0.47863513 0.         0.        ]
 [0.         0.2140333  0.7859667  0.         0.         0.        ]
 [0.         0.11984171 0.         0.         0.44007915 0.44007915]]


可以看到，l2归一化的平方和相加为1